In [1]:
import pickle as pkl
import numpy as np
import torch
from mmseg.datasets.cityscapes import CityscapesDataset
CLASSES = CityscapesDataset.CLASSES
import pandas as pd

/srv/flash1/hmaheshwari7/miniconda/envs/videoda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [3]:
with open("../../work_dirs/mmv5/invfn1_4gpu_gta_hrdargb_05-15-18-07-28/eval_results.pkl", "rb") as f:
    data = pkl.load(f)

In [4]:
i_data_dict = {k: v for k, v in data["cml_intersect"].items() if v.sum() != 0}
u_data_dict = {k: v for k, v in data["cml_union"].items() if v.sum() != 0}
iou_data_dict = {k: v / u_data_dict[k] for k, v in i_data_dict.items()}

pixel_correct_data_dict = {k: v for k, v in data["pixelwise_correct"].items() if v.sum() != 0}
pixel_total_data_dict = {k: v for k, v in data["pixelwise_total"].items() if v.sum() != 0}
pixel_acc_data_dict = {k: v / pixel_total_data_dict[k] for k, v in pixel_correct_data_dict.items()}

In [5]:
iou_data_dict["Class"] = CLASSES
pixel_acc_data_dict["Class"] = CLASSES

In [6]:
iou_df = pd.DataFrame.from_dict(iou_data_dict).set_index("Class").T
iou_df["Average"] = iou_df.mean(axis=1)
pixel_acc_df = pd.DataFrame.from_dict(pixel_acc_data_dict).set_index("Class").T
pixel_acc_df["Average"] = pixel_acc_df.mean(axis=1)

In [7]:
iou_df[["Average"]]

Class,Average
mIoU,0.640306
mIoU_gt_pred,0.643859
pred_pred,0.854801
gt_pred,0.631560
M5,0.663741
M5Fixed,0.670259


In [8]:
pixel_acc_df[["Average"]]

Class,Average
mIoU,0.753207
mIoU_gt_pred,0.754520
pred_pred,0.914863
gt_pred,0.787557
M5Fixed,0.772322


In [52]:
def get_metrics(confusion_matrix):
    confusion_matrix = pd.DataFrame(confusion_matrix)
    confusion_matrix.index = CLASSES
    confusion_matrix.columns = CLASSES
    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)

    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    metrics = {"FP": FP, 
                "FN": FN,
                "TP": TP,
                "TN": TN,
                "TPR": TPR,
                "TNR": TNR,
                "PPV": PPV,
                "NPV": NPV,
                "FPR": FPR,
                "FNR": FNR,
                "FDR": FDR,
                "ACC": ACC}
    return metrics


def get_normalized_cf(confusion_matrix):
    confusion_matrix /= confusion_matrix.sum(axis=1, keepdims=True)
    confusion_matrix = pd.DataFrame(confusion_matrix)
    confusion_matrix.index = CLASSES
    confusion_matrix.columns = CLASSES
    return (confusion_matrix).round(3)


In [53]:
metrics = get_metrics(data['confusion_matrix']['mIoU'].numpy())
cf = get_normalized_cf(data['confusion_matrix']['mIoU'].numpy())